In [ ]:
# Polarization CLEAN in a bottle
______________________________________________________________________________

Algorithm: Pratley, Luke and Johnston-Hollit Melanie, An improved method for 
polarimetric image restoration in interferometry, 2016

Implementation (C) B Hugo, SKA-SA

Credit: L Bester's Hogbom CLEAN implementation, SKA-SA
______________________________________________________________________________

In [ ]:
import math as mp
from numpy import *
import numpy as np
import matplotlib.pyplot as plt
from pyrap.tables import table as tbl
import scipy.signal
import time
from IPython.display import clear_output
from IPython.html.widgets import FloatProgress
from IPython.display import display
from mpl_toolkits.axes_grid1 import AxesGrid
from scipy import optimize as opt
from mpl_toolkits.axes_grid1 import ImageGrid
%matplotlib inline

In [ ]:
#(C) Landman Bester
def twoD_Gaussian((x, y), amplitude, xo, yo, sigma_x, sigma_y, theta, offset):
    xo = float(xo)
    yo = float(yo)
    a = (cos(theta)**2)/(2*sigma_x**2) + (sin(theta)**2)/(2*sigma_y**2)
    b = -(sin(2*theta))/(4*sigma_x**2) + (sin(2*theta))/(4*sigma_y**2)
    c = (sin(theta)**2)/(2*sigma_x**2) + (cos(theta)**2)/(2*sigma_y**2)
    g = offset + amplitude*exp( - (a*((x-xo)**2) + 2*b*(x-xo)*(y-yo) + c*((y-yo)**2)))
    return g.flatten()

#(C) Landman Bester
def fit_2D_Gaussian(PSF):
    """
    Fit an elliptical Gaussian to the primary lobe of the PSF
    """
    #Get the full width at half maximum height of the PSF
    I = argwhere(PSF>=0.5*PSF.max())
    #Create an array with these values at the same indices and zeros otherwise
    lk,mk = PSF.shape
    psf_fit = zeros([lk,mk])
    psf_fit[I[:,0],I[:,1]] = PSF[I[:,0],I[:,1]]
    # Create x and y indices
    x = linspace(0, PSF.shape[0]-1, PSF.shape[0])
    y = linspace(0, PSF.shape[1]-1, PSF.shape[1])
    x, y = meshgrid(x, y)
    # Set starting point of optimiser
    initial_guess = (0.5,lk/2,mk/2,1.75,1.4,-4.0,0)
    #Flatten the data
    data = psf_fit.ravel()
    #Fit the function (Gaussian for now)
    popt, pcov = opt.curve_fit(twoD_Gaussian, (x, y), data, p0=initial_guess)
    #Get function with fitted params
    data_fitted = twoD_Gaussian((x, y), *popt)
    #Normalise the psf to have a max value of one
    data_fitted = data_fitted/data_fitted.max()
    return data_fitted.reshape(lk,mk)


In [ ]:
def find_peak(Iresidue,Qresidue,Uresidue,Vresidue,mode="I"):
    """
    Finds peak in image or combination of images depending on mode
    Q+iU: Luke Pratley's joint linear (P) polarization clean
    I: Intensity only
    """
    if mode == "Q+iU":
        values_map = Qresidue + Uresidue*1.0j
        peakmap = (np.abs(values_map))**2
        p,q = (argwhere(peakmap==peakmap.max())[0]).squeeze()
        pmin,qmin = (argwhere(peakmap==peakmap.min())[0]).squeeze()
        Istar = values_map[p,q] #in Q and U the values may be negative or positive
    elif mode == "Q":
        peakmap = abs(Qresidue)
        p,q = (argwhere(peakmap==peakmap.max())[0]).squeeze()
        pmin,qmin = (argwhere(peakmap==peakmap.min())[0]).squeeze()
        Istar = Qresidue[p,q]
    elif mode == "U":
        peakmap = abs(Uresidue)
        p,q = (argwhere(peakmap==peakmap.max())[0]).squeeze()
        pmin,qmin = (argwhere(peakmap==peakmap.min())[0]).squeeze()
        Istar = Uresidue[p,q]
    elif mode == "V":
        peakmap = abs(Vresidue)
        p,q = (argwhere(peakmap==peakmap.max())[0]).squeeze()
        pmin,qmin = (argwhere(peakmap==peakmap.min())[0]).squeeze()
        Istar = Vresidue[p,q]
    elif mode == "I":
        peakmap = Iresidue
        p,q = (argwhere(peakmap==peakmap.max())[0]).squeeze()
        pmin,qmin = (argwhere(peakmap==peakmap.min())[0]).squeeze()
        Istar = Iresidue[p,q]
    else:
        raise ValueError("Peakfinding only works for mode one of [Q+iU, I]")
    return p,q,pmin,qmin,Istar

def build_cleanmap(ICLEAN,QCLEAN,UCLEAN,VCLEAN,Istar,gamma,p,q,mode="I"):
    if mode == "Q+iU":
        QCLEAN[p,q] += Istar.real*gamma
        UCLEAN[p,q] += Istar.imag*gamma
    elif mode == "Q":
        QCLEAN[p,q] += Istar*gamma
    elif mode == "U":
        UCLEAN[p,q] += Istar*gamma
    elif mode == "V":
        VCLEAN[p,q] += Istar*gamma
    elif mode == "I":
        ICLEAN[p,q] += Istar*gamma
    else:
        raise ValueError("Cleanmap building only works for mode one of [Q+iU, I]")
        
def update_residual(Iresidue,Qresidue,Uresidue,Vresidue,Istar,gamma,p,q,PSF,mode="I"):
    if mode == "Q+iU":
        npix = Iresidue.shape[0] #Assuming square image
        Qresidue -= gamma*Istar.real*PSF[npix - p:2*npix - p,
                                         npix - q:2*npix - q] 
        Uresidue -= gamma*Istar.imag*PSF[npix - p:2*npix - p,
                                         npix - q:2*npix - q]
    elif mode == "Q":
        npix = Iresidue.shape[0] #Assuming square image
        Qresidue -= gamma*Istar*PSF[npix - p:2*npix - p,
                                    npix - q:2*npix - q] 
    elif mode == "U":
        npix = Iresidue.shape[0] #Assuming square image
        Uresidue -= gamma*Istar*PSF[npix - p:2*npix - p,
                                    npix - q:2*npix - q] 
    elif mode == "V":
        npix = Iresidue.shape[0] #Assuming square image
        Vresidue -= gamma*Istar*PSF[npix - p:2*npix - p,
                                    npix - q:2*npix - q] 
    elif mode == "I":
        npix = Iresidue.shape[0] #Assuming square image
        Iresidue -= gamma*Istar*PSF[npix - p:2*npix - p,
                                    npix - q:2*npix - q] 
    else:
        raise ValueError("Residual subtraction only works for mode one of [Q+iU, I]")
        
def convolve_model(CLEAN_BEAM,ICLEAN,QCLEAN,UCLEAN,VCLEAN,mode):
    ICONV_MODEL = np.zeros(ICLEAN.shape)
    QCONV_MODEL = np.zeros(ICLEAN.shape)
    UCONV_MODEL = np.zeros(ICLEAN.shape)
    VCONV_MODEL = np.zeros(ICLEAN.shape)
    if mode == "Q+iU":
        QCONV_MODEL[:,:] = scipy.signal.fftconvolve(QCLEAN,CLEAN_BEAM,mode='same') #, cval=0.0) #Fast using fft
        UCONV_MODEL[:,:] = scipy.signal.fftconvolve(UCLEAN,CLEAN_BEAM,mode='same') #, cval=0.0) #Fast using fft
    elif mode == "Q":
        QCONV_MODEL[:,:] = scipy.signal.fftconvolve(QCLEAN,CLEAN_BEAM,mode='same') #, cval=0.0) #Fast using fft
    elif mode == "U":
        UCONV_MODEL[:,:] = scipy.signal.fftconvolve(UCLEAN,CLEAN_BEAM,mode='same') #, cval=0.0) #Fast using fft
    elif mode == "V":
        VCONV_MODEL[:,:] = scipy.signal.fftconvolve(VCLEAN,CLEAN_BEAM,mode='same') #, cval=0.0) #Fast using fft
    elif mode == "I":
        ICONV_MODEL[:,:] = scipy.signal.fftconvolve(ICLEAN,CLEAN_BEAM,mode='same') #, cval=0.0) #Fast using fft
    else:
        raise ValueError("Convolve model only works for mode one of [Q+iU, I]")
    return ICONV_MODEL, QCONV_MODEL, UCONV_MODEL, VCONV_MODEL

def CLEAN_HOG(IDIRTY,QDIRTY,UDIRTY,VDIRTY,
              PSF,gamma = 0.1,threshold = "Default", niter = "Default", 
              plot_on=True, mode="I"):
    """
    This is Hogbom CLEAN assuming a full cleaning window
    Input: IDirty = the dirty image to be cleaned
    PSF = the point spread function
    gamma = the gain factor (must be less than one)
    theshold = the threshold to clean up to
    niter = the maximum number of iterations allowed
    Output: ICLEAN = Clean model, IRESIDUAL = Residuals, 
            IRESTORE = Restored map, ICONV_CLEAN = Clean model convolved with clean beam
    
    """
    #deep copy dirties to first residues, want to keep the original dirty maps
    ID = np.copy(IDIRTY)
    QD = np.copy(QDIRTY) 
    UD = np.copy(UDIRTY) 
    VD = np.copy(VDIRTY) 
    #Check that all the dirty maps have the same shape
    if (ID.shape[0] != QD.shape[0] or QD.shape[0] != UD.shape[0] or UD.shape[0] != VD.shape[0] or
        ID.shape[1] != QD.shape[1] or QD.shape[1] != UD.shape[1] or UD.shape[1] != VD.shape[1]):
            raise ValueError("Polarized dirties must have the same shape")
    #Check that PSF is twice the size of ID
    if PSF.shape[0] != 2*ID.shape[0] or PSF.shape[1] != 2*ID.shape[1]:
        raise ValueError("Warning PSF not right size")
    #Initialise array to store cleaned image
    ICLEAN = zeros([npix,npix])
    QCLEAN = zeros([npix,npix])
    UCLEAN = zeros([npix,npix])
    VCLEAN = zeros([npix,npix])

    if niter == "Default":
        niter = 3*npix

    p,q,pmin,qmin,Istar = find_peak(ID,QD,UD,VD,mode)
    if threshold=="Default":
        threshold = 0.2*np.abs(Istar) #Imin + 0.001*(Istar - Imin)
        print "Threshold set at ", threshold
    else:
        print "Assuming user set threshold"

    #CLEAN the image
    i = 0 #counter index
    pbar_clean = FloatProgress(min=0, max=100)
    display(pbar_clean)

    while np.abs(Istar) > threshold and i <= niter:
        #First we set the
        build_cleanmap(ICLEAN,QCLEAN,UCLEAN,VCLEAN,Istar,gamma,p,q,mode)
        #Subtract out pixel
        update_residual(ID,QD,UD,VD,Istar,gamma,p,q,PSF,mode)
        #Get new indices where ID is max
        p,q,_,_,Istar = find_peak(ID,QD,UD,VD,mode)
        #Increment counter
        i += 1
        #Warn if niter exceeded
        if i > niter:
            print "Warning: number of iterations exceeded"
            print "Minimum ID = ", ID.max()
        pbar_clean.value = i / float(niter) * 100.0
    pbar_clean.value = 100 # done
    print "Done cleaning for mode %s after %d iterations. Now restoring..." % (mode,i)
    #get the ideal beam (fit 2D Gaussian to HWFH of PSF)
    CLEAN_BEAM = fit_2D_Gaussian(PSF)
    
    #Now convolve ICLEAN with ideal beam
    ICONV_MODEL, QCONV_MODEL, UCONV_MODEL, VCONV_MODEL = convolve_model(CLEAN_BEAM, ICLEAN, QCLEAN, UCLEAN, VCLEAN, mode)
    
    #Finally we add the residuals back to the image
    IRESIDUE = ID
    QRESIDUE = QD
    URESIDUE = UD
    VRESIDUE = VD
    
    IRESTORE = ICLEAN #ICONV_MODEL + IRESIDUE
    QRESTORE = QCONV_MODEL + QRESIDUE
    URESTORE = UCONV_MODEL + URESIDUE
    VRESTORE = VCONV_MODEL + VRESIDUE
    
    return (ICLEAN, IRESIDUE, IRESTORE, ICONV_MODEL,QCLEAN, QRESIDUE, QRESTORE, QCONV_MODEL,
            UCLEAN, URESIDUE, URESTORE, UCONV_MODEL,VCLEAN, VRESIDUE, VRESTORE, VCONV_MODEL)

In [ ]:
class AA_filter:
    """
    Anti-Aliasing filter
    
    Keyword arguments for __init__:
    filter_half_support --- Half support (N) of the filter; the filter has a full support of N*2 + 1 taps
    filter_oversampling_factor --- Number of spaces in-between grid-steps (improves gridding/degridding accuracy)
    filter_type --- box (nearest-neighbour), sinc or gaussian_sinc
    """
    half_sup = 0
    oversample = 0
    full_sup_wo_padding = 0
    full_sup = 0
    no_taps = 0
    filter_taps = None
    def __init__(self, filter_half_support, filter_oversampling_factor, filter_type):
        self.half_sup = filter_half_support
        self.oversample = filter_oversampling_factor
        self.full_sup_wo_padding = (filter_half_support * 2 + 1)
        self.full_sup = self.full_sup_wo_padding + 2 #+ padding
        self.no_taps = self.full_sup + (self.full_sup - 1) * (filter_oversampling_factor - 1)
        taps = np.arange(self.no_taps)/float(filter_oversampling_factor) - self.full_sup / 2
        if filter_type == "box":
            self.filter_taps = np.where((taps >= -0.5) & (taps <= 0.5),
                                        np.ones([len(taps)]),np.zeros([len(taps)]))
        elif filter_type == "sinc":
            self.filter_taps = np.sinc(taps)
        elif filter_type == "gaussian_sinc":
            alpha_1=1.55
            alpha_2=2.52
            self.filter_taps = np.sin(np.pi/alpha_1*(taps+0.00000000001))/(np.pi*(taps+0.00000000001))*np.exp(-(taps/alpha_2)**2)
        else:
            raise ValueError("Expected one of 'box','sinc' or 'gausian_sinc'")

In [ ]:
class grid_it(object):
    
    def __init__(self,uvw,ref_lda,Nx,Ny,convolution_filter):
        self.uvw = uvw
        self.ref_lda = ref_lda
        self.Nx = Nx
        self.Ny = Ny
        self.convolution_filter = convolution_filter
        
    def give_IR(self,vis):
        filter_index = np.arange(-self.convolution_filter.half_sup,self.convolution_filter.half_sup+1)
        measurement_regular = np.zeros([vis.shape[1],self.Ny,self.Nx],dtype=np.complex) #one grid for the resampled visibilities

        for r in range(0,uvw.shape[0]):
            for c in range(vis.shape[1]):
                scaled_uv = self.uvw[r,:] / self.ref_lda[c] 
                disc_u = int(scaled_uv[0])
                disc_v = int(scaled_uv[1])
                frac_u_offset = int((self.convolution_filter.half_sup + 1 + (-scaled_uv[0] + disc_u)) * self.convolution_filter.oversample)
                frac_v_offset = int((self.convolution_filter.half_sup + 1 + (-scaled_uv[1] + disc_v)) * self.convolution_filter.oversample)
                if (disc_v + self.Ny // 2 + self.convolution_filter.half_sup >= self.Ny or 
                    disc_u + self.Nx // 2 + self.convolution_filter.half_sup >= self.Nx or
                    disc_v + self.Ny // 2 - self.convolution_filter.half_sup < 0 or 
                    disc_u + self.Nx // 2 - self.convolution_filter.half_sup < 0): 
                    continue
                for conv_v in filter_index:
                    v_tap = self.convolution_filter.filter_taps[conv_v * self.convolution_filter.oversample + frac_v_offset]  
                    grid_pos_v = disc_v + conv_v + self.Ny // 2
                    for conv_u in filter_index:
                        u_tap = self.convolution_filter.filter_taps[conv_u * self.convolution_filter.oversample + frac_u_offset]
                        conv_weight = v_tap * u_tap
                        grid_pos_u = disc_u + conv_u + self.Ny // 2
                        measurement_regular[c,grid_pos_v,grid_pos_u] += vis[r,c] * conv_weight              
        dirty = np.zeros(measurement_regular.shape,dtype=measurement_regular.dtype)
        for c in range(vis.shape[1]):
            dirty[c,:,:] = np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(measurement_regular[c,:,:])))
        return dirty

In [ ]:
def grid_ifft(vis,uvw,ref_lda,Nx,Ny,convolution_filter):
    """
    Convolutional gridder (continnuum)
    
    Keyword arguments:
    vis --- Visibilities as sampled by the interferometer
    uvw --- interferometer's scaled uvw coordinates. (Prerequisite: these uv points are already scaled by the simularity
            theorem, such that -N_x*Cell_l*0.5 <= theta_l <= N_x*Cell_l*0.5 and
            -N_y*Cell_m*0.5 <= theta_m <= N_y*Cell_m*0.5
    ref_lda --- array of reference lambdas (size of vis channels)
    Nx,Ny --- size of image in pixels
    convolution_filter --- pre-instantiated AA_filter anti-aliasing filter object
    """
    assert vis.shape[1] == ref_lda.shape[0], (vis.shape[1], ref_lda.shape[0])
    filter_index = np.arange(-convolution_filter.half_sup,convolution_filter.half_sup+1)
    measurement_regular = np.zeros([vis.shape[1],vis.shape[2],Ny,Nx],dtype=np.complex) #one grid for the resampled visibilities
    #for deconvolution the PSF should be 2x size of the image (see Hogbom CLEAN for details):
    sampling_regular = np.zeros([vis.shape[1],2*Ny,2*Nx],dtype=np.complex) #one grid for the resampled sampling function
    
    pbar = FloatProgress(min=0, max=100)
    display(pbar)
    for r in range(0,uvw.shape[0]):
        pbar.value = r / float(uvw.shape[0]) * 100
        for c in range(vis.shape[1]):
            scaled_uv = uvw[r,:] / ref_lda[c] 
            disc_u = int(scaled_uv[0])
            disc_v = int(scaled_uv[1])
            frac_u_offset = int((convolution_filter.half_sup + 1 + (-scaled_uv[0] + disc_u)) * convolution_filter.oversample)
            frac_v_offset = int((convolution_filter.half_sup + 1 + (-scaled_uv[1] + disc_v)) * convolution_filter.oversample)
            disc_u_psf = int(scaled_uv[0]*2)
            disc_v_psf = int(scaled_uv[1]*2)
            frac_u_offset_psf = int((convolution_filter.half_sup + 1 + (-scaled_uv[0]*2 + disc_u_psf)) * convolution_filter.oversample)
            frac_v_offset_psf = int((convolution_filter.half_sup + 1 + (-scaled_uv[1]*2 + disc_v_psf)) * convolution_filter.oversample)
            if (disc_v + Ny // 2 + convolution_filter.half_sup >= Ny or 
                disc_u + Nx // 2 + convolution_filter.half_sup >= Nx or
                disc_v + Ny // 2 - convolution_filter.half_sup < 0 or 
                disc_u + Nx // 2 - convolution_filter.half_sup < 0): 
                continue
            for conv_v in filter_index:
                v_tap = convolution_filter.filter_taps[conv_v * convolution_filter.oversample + frac_v_offset]  
                v_tap_psf = convolution_filter.filter_taps[conv_v * convolution_filter.oversample + frac_v_offset_psf]  
                grid_pos_v = disc_v + conv_v + Ny // 2
                grid_pos_v_psf = disc_v_psf + conv_v + Ny
                for conv_u in filter_index:
                    u_tap = convolution_filter.filter_taps[conv_u * convolution_filter.oversample + frac_u_offset]
                    u_tap_psf = convolution_filter.filter_taps[conv_u * convolution_filter.oversample + frac_u_offset_psf]
                    conv_weight = v_tap * u_tap
                    conv_weight_psf = v_tap_psf * u_tap_psf
                    grid_pos_u = disc_u + conv_u + Ny // 2
                    grid_pos_u_psf = disc_u_psf + conv_u + Nx
                    for p in range(vis.shape[2]):
                        measurement_regular[c,p,grid_pos_v,grid_pos_u] += vis[r,c,p] * conv_weight
                    sampling_regular[c,grid_pos_v_psf,grid_pos_u_psf] += (1+0.0j) * conv_weight_psf
                    
    dirty = np.zeros(measurement_regular.shape,dtype=measurement_regular.dtype)
    psf = np.zeros(sampling_regular.shape,dtype=sampling_regular.dtype)
    
    for c in range(vis.shape[1]):
        for p in range(vis.shape[2]):
            dirty[c,p,:,:] = np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(measurement_regular[c,p,:,:])))
        psf[c,:,:] = np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(sampling_regular[c,:,:])))
    return dirty,psf

In [ ]:
#Collection of DFT routines for accuracy
def get_lmn(l,m):
    """
    Get (l,m,n) triple. uvw is a tuple containg u, v and w coordinates for the measurement
    set and l and m are coordinates on the sky.
    """
    #First get n for each pair of l and m
    Nx = l.size #numbere of pixels in l
    Ny = m.size #number of pixels in m
    n = np.zeros(Nx*Ny) #array to store
    lmn = np.zeros([Nx*Ny,3]) #array to store combos of l, m and n
    for i in range(Nx):
        for j in range(Ny):
            n[i*Ny + j] = np.sqrt(1.0 - l[i]**2 - m[j]**2)
            lmn[i*Ny + j,0] = l[i]
            lmn[i*Ny + j,1] = m[j]
            lmn[i*Ny + j,2] = n[i*Ny+j] - 1
    return lmn

def set_kernels(Nx,Ny,lmn,uvw,ref_lambda):
    """
    Sets the DFT kernels for each channel
    """
    Nch = ref_lambda.size
    Nv = uvw.shape[0]
    tmp = -2.0j*np.pi*dot(uvw,lmn.T)
    K = np.zeros([Nch,Nv, Nx*Ny],dtype=complex)
    #invK = np.zeros([Nch,Nx*Ny,Nv],dtype=complex)
    for k in range(Nch):
            K[k,:,:] = np.exp(tmp/ref_lambda[k])
            #invK[k,:,:] = np.exp(-tmp.T/ref_lambda[k])
    return K #invK

def DFT(I,K,n):
    tmp = np.zeros([K.shape[0],K.shape[1]],dtype=complex)
    for i in range(K.shape[0]):
        tmp[i,:] = np.dot(K[i,:,:],I[i,:]/n)
    return tmp

def iDFT(vis,invK,n):
    tmp = np.zeros([invK.shape[0],invK.shape[1]],dtype=complex)
    for i in range(invK.shape[0]):
        tmp[i,:] = n*np.dot(invK[i,:,:],vis[:,i])    
    return tmp

def flatten_ID(I):
    Nx = I.shape[1]
    Ny = I.shape[2]
    Iflat = np.zeros([1,Nx*Ny])
    for i in range(Nx):
        for j in range(Ny):
            Iflat[0,i*Ny + j] = I[0,i,j]
    return Iflat

# Simulate sky

In [ ]:
#import subprocess
#import Cattery.Siamese 
#turbo_sim_path = "%s/turbo-sim.py" % Cattery.Siamese.__path__[0]
#subprocess.check_call("cd DATA ; makems pols.makems.parset ; meqtree-pipeliner.py -c polarized_sky.tdl --mt=4 @pol %s =simulate" % turbo_sim_path, shell=True)

# Compute Dirty

In [ ]:
#Load in MS
ms = tbl("DATA/polarized_sky.MS_p0/")

#Get freq info
msfreq = tbl("DATA/polarized_sky.MS_p0::SPECTRAL_WINDOW")

#Get pointing center
msfield = tbl("DATA/polarized_sky.MS_p0::FIELD")
ra0, dec0 = msfield.getcol("PHASE_DIR").squeeze() #in radians

#Set AA filter
aa = AA_filter(3,63,"sinc")

#Some constants and conversion factors
c = 2.99792458e8 #speed of light
npix = 128 #1025
ARCSEC2RAD = 4.8481e-6
delta_pix = 2 * ARCSEC2RAD * 2 * 2 * 2
uv_scale = npix * delta_pix

#Select subset of data
nchan = 5 #-1 to select all (NOTE this selects all but the last)
nrows = 1000 #-1 to select all
uvw = ms.getcol("UVW")[0:nrows,:] 
vis = ms.getcol("DATA")[0:nrows,0:nchan,:]
#Delete autocorelations
I = np.argwhere(uvw[:,0] == 0.0).squeeze()
uvw = np.delete(uvw,I,axis=0)
scaled_uvw = uvw * uv_scale
vis = np.delete(vis,I,axis=0)

#Get frequencies
ref_freq = msfreq.getcol("CHAN_FREQ").squeeze()[0:nchan]

#Close tables
ms.close()
msfreq.close()
msfield.close()

#Get wavelengths
ref_lambda = c / ref_freq

#Compute dirty using gridder
print "Gridding"
dirty,psf = grid_ifft(vis,scaled_uvw,ref_lambda,npix,npix,aa)

#Get ID from gridder for comparison
ID2 = np.real((dirty[:,0,:,:]+dirty[:,3,:,:])*0.5)/4.0

psf2 = np.abs(psf)

#Normalise ID and PSF
#for i in range(nchan):
#    ID2[i,:,:] /= (np.max(psf2[i,:,:])*4) #x4 because the N**2 FFT normalization factor on a square image doubles the size
#    psf2[i,:,:] /= np.max(psf2[i,:,:])

In [ ]:
#Here we get ID and PSF using DFT
#Set ra and dec
ra = ra0 + np.linspace(ra0 - npix*delta_pix/2.0,ra0 + npix*delta_pix/2.0,npix)
ra_PSF  = ra0 + np.linspace(ra0 - npix*delta_pix,ra0 + npix*delta_pix,2*npix)
delta_ra = ra - ra0
delta_ra_PSF = ra_PSF - ra0
dec = dec0 + np.linspace(dec0 - npix*delta_pix/2.0,dec0 + npix*delta_pix/2.0,npix)
dec_PSF = dec0 + np.linspace(dec0 - npix*delta_pix,dec0 + npix*delta_pix,2*npix)

#Get corresponding l and m
l = (np.cos(dec)*np.sin(delta_ra))
l_PSF = (np.cos(dec_PSF)*np.sin(delta_ra_PSF))
m = (-np.sin(dec)*np.cos(dec0) - np.cos(dec)*np.sin(dec0)*cos(delta_ra))
m_PSF = (-np.sin(dec_PSF)*np.cos(dec0) - np.cos(dec_PSF)*np.sin(dec0)*cos(delta_ra_PSF))

print "Getting lmn"
lmn = get_lmn(l,m)
n = lmn[:,2] + 1.0
lmn_PSF = get_lmn(l_PSF,m_PSF)
n_PSF = lmn_PSF[:,2] + 1.0

print "Getting DFT kernels for PSF"
invK_PSF = set_kernels(2*npix,2*npix,lmn_PSF,uvw,ref_lambda)

print "Computing PSF"
PSF = iDFT(np.ones([vis.shape[0],nchan]),invK_PSF,n_PSF) #np.abs(dot(invK[0,:,:],np.ones(vis.shape[0])))
PSF = np.abs(PSF.reshape(nchan,2*npix,2*npix))

del(invK_PSF)
del(l_PSF)
del(m_PSF)
del(lmn_PSF)
del(n_PSF)

print "Getting DFT kernels"
invK = set_kernels(npix,npix,lmn,uvw,ref_lambda)
K = np.zeros([nchan,uvw.shape[0],npix*npix])
for i in range(nchan):
    K[i,:,:] = invK[i,:,:].conjugate().T

print "Doing iDFT"
visI = (vis[:,:,0] + vis[:,:,3])*0.5
ID = np.real(iDFT(visI,invK,n))

IDt = ID.copy()

print "Doing DFT"
visI_pred = DFT(ID,K,n)

#Reshape into image
ID = ID.reshape(nchan,npix,npix)

#Normalise ID and PSF
for i in range(nchan):
    ID[i,:,:] /= (np.max(PSF[i,:,:])) #x4 because the N**2 FFT normalization factor on a square image doubles the size
    PSF[i,:,:] /= np.max(PSF[i,:,:])

#print np.allclose(visI,visI_pred,rtol=1e-03, atol=1e-03)

In [ ]:
ch = 0
#Plot results for comparison
Fig = plt.figure(1,(15,15))
grid = ImageGrid(Fig, 111,  # similar to subplot(111)
                nrows_ncols=(1, 2),
                direction="row",
                axes_pad=0.5,
                add_all=True,
                label_mode="1",
                share_all=True,
                cbar_location="right",
                cbar_mode="each",
                cbar_size="3%")

grid[0].set_title("ID_DFT")
im = grid[0].imshow(np.fliplr(ID[ch,:,:]).T, interpolation="nearest", cmap="cubehelix")
grid[0].cax.colorbar(im)
grid[1].set_title("ID_grid")
im = grid[1].imshow(ID2[ch,:,:], interpolation="nearest", cmap="cubehelix")
grid[1].cax.colorbar(im)

In [ ]:
#Plot results for comparison
Fig = plt.figure(1,(15,15))
grid = ImageGrid(Fig, 111,  # similar to subplot(111)
                nrows_ncols=(1, 2),
                direction="row",
                axes_pad=0.5,
                add_all=True,
                label_mode="1",
                share_all=True,
                cbar_location="right",
                cbar_mode="each",
                cbar_size="3%")


grid[0].set_title("PSF_DFT")
im = grid[0].imshow(np.fliplr(PSF[ch,:,:]).T, interpolation="nearest", cmap="cubehelix")
grid[0].cax.colorbar(im)
grid[1].set_title("PSF_grid")
im = grid[1].imshow(np.abs(psf2[ch,:,:]), interpolation="nearest", cmap="cubehelix")
grid[1].cax.colorbar(im)
plt.show()

# Correlations -> Stokes

In [ ]:
I = np.real((dirty[:,0,:,:]+dirty[:,3,:,:])*0.5)
Q = np.real((dirty[:,0,:,:]-dirty[:,3,:,:])*0.5)
U = np.real((dirty[:,1,:,:]+dirty[:,2,:,:])*0.5)
V = np.imag((dirty[:,1,:,:]-dirty[:,2,:,:])*0.5)

#assume psf is the same for all polarizations:
I_PSF = np.abs(psf)
Q_PSF = np.abs(psf)
U_PSF = np.abs(psf)
V_PSF = np.abs(psf)

#Set normalisation for each channel
for i in range(nchan):
    I[i,:,:] /= (np.max(I_PSF[i,:,:])*4) #x4 because the N**2 FFT normalization factor on a square image doubles the size
    Q[i,:,:] /= (np.max(Q_PSF[i,:,:])*4)
    U[i,:,:] /= (np.max(U_PSF[i,:,:])*4)
    V[i,:,:] /= (np.max(V_PSF[i,:,:])*4)
    I_PSF[i,:,:] /= np.max(I_PSF[i,:,:])
    Q_PSF[i,:,:] /= np.max(Q_PSF[i,:,:])
    U_PSF[i,:,:] /= np.max(U_PSF[i,:,:])
    V_PSF[i,:,:] /= np.max(V_PSF[i,:,:])

# Dirty map

In [ ]:
ch = 0
F = plt.figure(1,(15,15))
grid = ImageGrid(F, 111,  # similar to subplot(111)
                nrows_ncols=(2, 2),
                direction="row",
                axes_pad=0.5,
                add_all=True,
                label_mode="1",
                share_all=True,
                cbar_location="right",
                cbar_mode="each",
                cbar_size="3%")

grid[0].set_title("I")
im = grid[0].imshow(I[ch,:,:], interpolation="nearest", cmap="cubehelix")
grid[0].cax.colorbar(im)
grid[1].set_title("Q")
im = grid[1].imshow(Q[ch,:,:], interpolation="nearest", cmap="cubehelix")
grid[1].cax.colorbar(im)
grid[2].set_title("U")
im = grid[2].imshow(U[ch,:,:], interpolation="nearest", cmap="cubehelix")
grid[2].cax.colorbar(im)
grid[3].set_title("V")
im = grid[3].imshow(V[ch,:,:], interpolation="nearest", cmap="cubehelix")
grid[3].cax.colorbar(im)
plt.show()

# PSF

In [ ]:
ch = 0
F = plt.figure(1,(15,15))
grid = ImageGrid(F, 111,  # similar to subplot(111)
                nrows_ncols=(2, 2),
                direction="row",
                axes_pad=0.5,
                add_all=True,
                label_mode="1",
                share_all=True,
                cbar_location="right",
                cbar_mode="each",
                cbar_size="3%")

grid[0].set_title("I PSF")
im = grid[0].imshow(I_PSF[ch], interpolation="nearest", cmap="cubehelix")
grid[0].cax.colorbar(im)
grid[1].set_title("Q PSF")
im = grid[1].imshow(Q_PSF[ch], interpolation="nearest", cmap="cubehelix")
grid[1].cax.colorbar(im)
grid[2].set_title("U PSF")
im = grid[2].imshow(U_PSF[ch], interpolation="nearest", cmap="cubehelix")
grid[2].cax.colorbar(im)
grid[3].set_title("V PSF")
im = grid[3].imshow(V_PSF[ch], interpolation="nearest", cmap="cubehelix")
grid[3].cax.colorbar(im)
plt.show()

# Clean (per channel)

In [ ]:
ch = 0
print "Cleaning I..."
I_clean,I_residue,I_restored,I_convmodel, _, _, _, _, _, _, _, _ , _, _, _, _ = CLEAN_HOG(I[ch],Q[ch],U[ch],V[ch],I_PSF[ch],mode="I")
print "\t ...done"
print "Cleaning Q+Ui..."
_, _, _, _, Q_clean, Q_residue, Q_restored, Q_convmodel, U_clean, U_residue, U_restored, U_convmodel, _, _, _, _ = CLEAN_HOG(I[ch],Q[ch],U[ch],V[ch],I_PSF[ch],mode="Q+iU")
print "\t ...done"
print "Cleaning V..."
_, _, _, _, _, _, _, _, _, _, _, _, V_clean, V_residue, V_restored, V_convmodel = CLEAN_HOG(I[ch],Q[ch],U[ch],V[ch],I_PSF[ch],mode="V")
print "\t ...done"

## Cleaned I

In [ ]:
F = plt.figure(1,(15,15))
grid = ImageGrid(F, 111,  # similar to subplot(111)
                nrows_ncols=(2, 2),
                direction="row",
                axes_pad=0.5,
                add_all=True,
                label_mode="1",
                share_all=True,
                cbar_location="right",
                cbar_mode="each",
                cbar_size="3%")

grid[0].set_title("I Restored")
im = grid[0].imshow(I_restored, interpolation="nearest", cmap="cubehelix")
grid[0].cax.colorbar(im)
grid[1].set_title("I Residue")
im = grid[1].imshow(I_residue, interpolation="nearest", cmap="cubehelix")
grid[1].cax.colorbar(im)
grid[2].set_title("I Dirty")
im = grid[2].imshow(I[ch], interpolation="nearest", cmap="cubehelix")
grid[2].cax.colorbar(im)
grid[3].set_title("I convolved clean map")
im = grid[3].imshow(I_convmodel, interpolation="nearest", cmap="cubehelix")
grid[3].cax.colorbar(im)
plt.show()

## Cleaned Linear

In [ ]:
F = plt.figure(1,(15,15))
grid = ImageGrid(F, 111,  # similar to subplot(111)
                nrows_ncols=(2, 2),
                direction="row",
                axes_pad=0.5,
                add_all=True,
                label_mode="1",
                share_all=True,
                cbar_location="right",
                cbar_mode="each",
                cbar_size="3%")

grid[0].set_title("Q Restored")
im = grid[0].imshow(Q_restored, interpolation="nearest", cmap="cubehelix")
grid[0].cax.colorbar(im)
grid[1].set_title("Q Residue")
im = grid[1].imshow(Q_residue, interpolation="nearest", cmap="cubehelix")
grid[1].cax.colorbar(im)
grid[2].set_title("Q Dirty")
im = grid[2].imshow(Q[ch], interpolation="nearest", cmap="cubehelix")
grid[2].cax.colorbar(im)
grid[3].set_title("Q convolved clean map")
im = grid[3].imshow(Q_convmodel, interpolation="nearest", cmap="cubehelix")
grid[3].cax.colorbar(im)
plt.show()

F = plt.figure(1,(15,15))
grid = ImageGrid(F, 111,  # similar to subplot(111)
                nrows_ncols=(2, 2),
                direction="row",
                axes_pad=0.5,
                add_all=True,
                label_mode="1",
                share_all=True,
                cbar_location="right",
                cbar_mode="each",
                cbar_size="3%")

grid[0].set_title("U Restored")
im = grid[0].imshow(U_restored, interpolation="nearest", cmap="cubehelix")
grid[0].cax.colorbar(im)
grid[1].set_title("U Residue")
im = grid[1].imshow(U_residue, interpolation="nearest", cmap="cubehelix")
grid[1].cax.colorbar(im)
grid[2].set_title("U Dirty")
im = grid[2].imshow(U[ch], interpolation="nearest", cmap="cubehelix")
grid[2].cax.colorbar(im)
grid[3].set_title("U convolved clean map")
im = grid[3].imshow(U_convmodel, interpolation="nearest", cmap="cubehelix")
grid[3].cax.colorbar(im)
plt.show()

# Circular polarization

In [ ]:
F = plt.figure(1,(15,15))
grid = ImageGrid(F, 111,  # similar to subplot(111)
                nrows_ncols=(2, 2),
                direction="row",
                axes_pad=0.5,
                add_all=True,
                label_mode="1",
                share_all=True,
                cbar_location="right",
                cbar_mode="each",
                cbar_size="3%")

grid[0].set_title("V Restored")
im = grid[0].imshow(V_restored, interpolation="nearest", cmap="cubehelix")
grid[0].cax.colorbar(im)
grid[1].set_title("V Residue")
im = grid[1].imshow(V_residue, interpolation="nearest", cmap="cubehelix")
grid[1].cax.colorbar(im)
grid[2].set_title("V Dirty")
im = grid[2].imshow(V[ch], interpolation="nearest", cmap="cubehelix")
grid[2].cax.colorbar(im)
grid[3].set_title("V convolved clean map")
im = grid[3].imshow(V_convmodel, interpolation="nearest", cmap="cubehelix")
grid[3].cax.colorbar(im)
plt.show()

# Joint Polarization vs. Seperate Polarization CLEAN

Here I compare the Pratley and Johnston-Hollitt joint polarization CLEAN algorithm with standard seperate polarization CLEAN. First I keep the frame centred as-is (shown above), then I rotate the frame to simulate a different choice of frame for the feeds, clean the images produced and then derotate the image. The result should be the same as for the original frame.

In [ ]:
#Let P = Q + iU, then rotate each complex number by theta:
theta = np.deg2rad(65)
P = Q[ch] + 1.0j*U[ch]
RP = P * np.exp(1.0j * theta)
#joint clean on rotated frame:
_, _, _, _, RQ_clean, RQ_residue, RQ_restored, RQ_convmodel, RU_clean, RU_residue, RU_restored, RU_convmodel, _, _, _, _ = CLEAN_HOG(I[ch],RP.real,RP.imag,V[ch],I_PSF[ch],mode="Q+iU")
#unrotate the frame:
RPClean = RQ_convmodel + 1.0j*RU_convmodel
PClean = RPClean * np.exp(1.0j * (-theta))

plt.figure(1,(5,5))
plt.title("Luke's Joint polarization CLEAN")
p1 = plt.scatter(PClean.real,PClean.imag, marker="+", color='blue', label='Rotated frame')
p2 = plt.scatter(Q_convmodel,U_convmodel,  marker="x", color='red', label='Original frame')
plt.xlim([-1.2,1.2])
plt.ylim([-1.2,1.2])
plt.xlabel("$\Re[C_p]$ Jy/Beam")
plt.ylabel("$\Im[C_p]$ Jy/Beam")
plt.legend()
plt.show()

As shown in the paper the CLEANing is indpendent of the choice of reference frame for the feeds wrt. the sky.

In [ ]:
#First simulate Traditional seperate polarization CLEAN on the original unrotated frame:
_, _, _, _, TradQ_clean, TradQ_residue, TradQ_restored, TradQ_convmodel, _, _, _, _, _, _, _, _ = CLEAN_HOG(I[ch],Q[ch],U[ch],V[ch],I_PSF[ch],mode="Q")
_, _, _, _, _, _, _, _, TradU_clean, TradU_residue, TradU_restored, TradU_convmodel,_, _, _, _ = CLEAN_HOG(I[ch],Q[ch],U[ch],V[ch],I_PSF[ch],mode="U")

#Let P = Q + iU, then rotate each complex number by theta:
theta = np.deg2rad(65)
P = Q[ch] + 1.0j*U[ch]
RP = P * np.exp(1.0j * theta)
#traditional seperate clean on rotated frame:
_, _, _, _, RQ_clean, RQ_residue, RQ_restored, RQ_convmodel, _, _, _, _, _, _, _, _ = CLEAN_HOG(I[ch],RP.real,RP.imag,V[ch],I_PSF[ch],mode="Q")
_, _, _, _, _, _, _, _, RU_clean, RU_residue, RU_restored, RU_convmodel,_, _, _, _ = CLEAN_HOG(I[ch],RP.real,RP.imag,V[ch],I_PSF[ch],mode="U")

#unrotate the frame:
RPClean = RQ_convmodel + 1.0j*RU_convmodel
PClean = RPClean * np.exp(1.0j * (-theta))
%matplotlib inline
plt.figure(1,(5,5))
plt.title("Traditional seperate polarization CLEAN")
plt.scatter(np.real(PClean),np.imag(PClean), marker="+", color='blue', label='Rotated frame')
plt.scatter(TradQ_convmodel,TradU_convmodel,  marker="x", color='red', label='Original frame')
plt.xlim([-1.2,1.2])
plt.ylim([-1.2,1.2])
plt.xlabel("$\Re[C_p]$ Jy/Beam")
plt.ylabel("$\Im[C_p]$ Jy/Beam")
plt.legend()
plt.show()


This plot confirms that Traditional CLEAN is dependent on the Feed reference frame.

# Joint deconvolution

Here we do joint deconvolution by searching for the peak in the average image (over channels). For realistic data containing noise the averaging procedure should increase the signal to noise ratio.  

In [ ]:
def give_average_image(I,Q,U,V,mode):
    Im = np.zeros_like(I)
    Qm = np.zeros_like(Q)
    Um = np.zeros_like(U)
    Vm = np.zeros_like(V)
    if mode == "I":
        Im = np.mean(I,axis=0)
    else:
        raise ValueError("Joint deconvolution only implemented for mode I")
    return Im, Qm, Um, Vm

def give_peak(p,q,Iresidue,Qresidue,Uresidue,Vresidue,mode="I"):
    """
    Finds peak in image or combination of images depending on mode
    Q+iU: Luke Pratley's joint linear (P) polarization clean
    I: Intensity only
    """
    if mode == "Q+iU":
        values_map = Qresidue + Uresidue*1.0j
        Istar = values_map[:,p,q] #in Q and U the values may be negative or positive
    elif mode == "Q":
        Istar = Qresidue[:,p,q]
    elif mode == "U":
        Istar = Uresidue[:,p,q]
    elif mode == "V":
        Istar = Vresidue[:,p,q]
    elif mode == "I":
        Istar = Iresidue[:,p,q]
    else:
        raise ValueError("Mode not supported")
    return Istar

def build_cleanmap_joint(ICLEAN,QCLEAN,UCLEAN,VCLEAN,Istar,gamma,p,q,mode="I"):
    if mode == "Q+iU":
        QCLEAN[:,p,q] += Istar.real*gamma
        UCLEAN[:,p,q] += Istar.imag*gamma
    elif mode == "Q":
        QCLEAN[:,p,q] += Istar*gamma
    elif mode == "U":
        UCLEAN[:,p,q] += Istar*gamma
    elif mode == "V":
        VCLEAN[:,p,q] += Istar*gamma
    elif mode == "I":
        ICLEAN[:,p,q] += Istar*gamma
    else:
        raise ValueError("Cleanmap building only works for mode one of [Q+iU, I]")
        
def update_residual_joint(Iresidue,Qresidue,Uresidue,Vresidue,Istar,gamma,p,q,PSF,mode="I"):
    if mode == "Q+iU":
        npix = Iresidue.shape[1] #Assuming square image
        Qresidue -= gamma*Istar.real[:,np.newaxis,np.newaxis]*PSF[:,npix - p:2*npix - p,
                                         npix - q:2*npix - q] 
        Uresidue -= gamma*Istar.imag[:,np.newaxis,np.newaxis]*PSF[:,npix - p:2*npix - p,
                                         npix - q:2*npix - q]
    elif mode == "Q":
        npix = Iresidue.shape[1] #Assuming square image
        Qresidue -= gamma*Istar[:,np.newaxis,np.newaxis]*PSF[:,npix - p:2*npix - p,
                                    npix - q:2*npix - q] 
    elif mode == "U":
        npix = Iresidue.shape[1] #Assuming square image
        Uresidue -= gamma*Istar[:,np.newaxis,np.newaxis]*PSF[:,npix - p:2*npix - p,
                                    npix - q:2*npix - q] 
    elif mode == "V":
        npix = Iresidue.shape[1] #Assuming square image
        Vresidue -= gamma*Istar[:,np.newaxis,np.newaxis]*PSF[:,npix - p:2*npix - p,
                                    npix - q:2*npix - q] 
    elif mode == "I":
        npix = Iresidue.shape[1] #Assuming square image
        Iresidue -= gamma*Istar[:,np.newaxis,np.newaxis]*PSF[:,npix - p:2*npix - p,
                                    npix - q:2*npix - q] 
    else:
        raise ValueError("Mode not supported")

def CLEAN_JOINT(IDIRTY,QDIRTY,UDIRTY,VDIRTY,
              PSF,gamma = 0.1,threshold = "Default", niter = "Default", 
              plot_on=True, mode="I"):
    """
    This is Hogbom CLEAN assuming a full cleaning window
    Input: IDirty = the dirty image to be cleaned
    PSF = the point spread function
    gamma = the gain factor (must be less than one)
    theshold = the threshold to clean up to
    niter = the maximum number of iterations allowed
    Output: ICLEAN = Clean model, IRESIDUAL = Residuals, 
            IRESTORE = Restored map, ICONV_CLEAN = Clean model convolved with clean beam
    
    """
    nchan = PSF.shape[0]
    #deep copy dirties to first residues, want to keep the original dirty maps
    ID = np.copy(IDIRTY)
    QD = np.copy(QDIRTY) 
    UD = np.copy(UDIRTY) 
    VD = np.copy(VDIRTY)
    
    #Get mean
    IDm, QDm, UDm, VDm = give_average_image(ID,QD,UD,VD,mode)
    
    #Check that all the dirty maps have the same shape
    if (ID.shape[0] != QD.shape[0] or QD.shape[0] != UD.shape[0] or UD.shape[0] != VD.shape[0] or
        ID.shape[1] != QD.shape[1] or QD.shape[1] != UD.shape[1] or UD.shape[1] != VD.shape[1] or
        ID.shape[2] != QD.shape[2] or QD.shape[2] != UD.shape[2] or UD.shape[2] != VD.shape[2]):
            raise ValueError("Polarized dirties must have the same shape")
    
    #Check that PSF is twice the size of ID
    if PSF.shape[0] != ID.shape[0] or PSF.shape[1] != 2*ID.shape[1] or PSF.shape[2] != 2*ID.shape[2]:
        raise ValueError("Warning PSF not right size")
    
    #Initialise array to store cleaned image
    ICLEAN = zeros([nchan,npix,npix])
    QCLEAN = zeros([nchan,npix,npix])
    UCLEAN = zeros([nchan,npix,npix])
    VCLEAN = zeros([nchan,npix,npix])

    #Set max iterations
    if niter == "Default":
        niter = 3*npix

    #Find peak in average image
    p,q,pmin,qmin,Istarm = find_peak(IDm,QDm,UDm,VDm,mode)

    #Set threshold to clean up to (note applied to average map)
    if threshold=="Default":
        threshold = 0.2*np.abs(Istarm) #Imin + 0.001*(Istar - Imin)
        print "Threshold set at ", threshold
    else:
        print "Assuming user set threshold"

    #Get Istar for all freqs
    Istar = give_peak(p,q,ID,QD,UD,VD,mode)
    
    #CLEAN the image
    i = 0 #counter index
    pbar_clean = FloatProgress(min=0, max=100)
    display(pbar_clean)

    while np.abs(Istarm) > threshold and i <= niter:
        #First we set the
        build_cleanmap_joint(ICLEAN,QCLEAN,UCLEAN,VCLEAN,Istar,gamma,p,q,mode)
        #Subtract out pixel
        update_residual_joint(ID,QD,UD,VD,Istar,gamma,p,q,PSF,mode)
        #Get averages for peak finding
        IDm, QDm, UDm, VDm = give_average_image(ID,QD,UD,VD,mode)
        #Get new indices where average image is max
        p,q,_,_,Istarm = find_peak(IDm,QDm,UDm,VDm,mode)
        Istar = give_peak(p,q,ID,QD,UD,VD,mode)
        #Increment counter
        i += 1
        #Warn if niter exceeded
        if i > niter:
            print "Warning: number of iterations exceeded"
            print "Minimum ID = ", ID.max()
        pbar_clean.value = i / float(niter) * 100.0
    pbar_clean.value = 100 # done
    print "Done cleaning for mode %s after %d iterations. Now restoring..." % (mode,i)
    
    ICONV_MODEL = np.zeros([nchan,npix,npix])
    QCONV_MODEL = np.zeros([nchan,npix,npix])
    UCONV_MODEL = np.zeros([nchan,npix,npix])
    VCONV_MODEL = np.zeros([nchan,npix,npix])
    IRESIDUE = ID
    QRESIDUE = QD
    URESIDUE = UD
    VRESIDUE = VD
    IRESTORE = np.zeros([nchan,npix,npix])
    QRESTORE = np.zeros([nchan,npix,npix])
    URESTORE = np.zeros([nchan,npix,npix])
    VRESTORE = np.zeros([nchan,npix,npix])
    
    for i in range(nchan):
        #get the ideal beam (fit 2D Gaussian to HWFH of PSF)
        CLEAN_BEAM = fit_2D_Gaussian(PSF[i,:,:])

        #Now convolve ICLEAN with ideal beam
        ICONV_MODEL[i,:,:], QCONV_MODEL[i,:,:], UCONV_MODEL[i,:,:], VCONV_MODEL[i,:,:] = convolve_model(CLEAN_BEAM, ICLEAN[i,:,:], QCLEAN[i,:,:], UCLEAN[i,:,:], VCLEAN[i,:,:], mode)

        #Finally we add the residuals back to the image

        IRESTORE[i,:,:] = ICONV_MODEL[i,:,:] + IRESIDUE[i,:,:]
        QRESTORE[i,:,:] = QCONV_MODEL[i,:,:] + QRESIDUE[i,:,:]
        URESTORE[i,:,:] = UCONV_MODEL[i,:,:] + URESIDUE[i,:,:]
        VRESTORE[i,:,:] = VCONV_MODEL[i,:,:] + VRESIDUE[i,:,:]
    
    return (ICLEAN, IRESIDUE, IRESTORE, ICONV_MODEL,QCLEAN, QRESIDUE, QRESTORE, QCONV_MODEL,
            UCLEAN, URESIDUE, URESTORE, UCONV_MODEL,VCLEAN, VRESIDUE, VRESTORE, VCONV_MODEL)

In [ ]:
print "Cleaning I..."
I_clean_j,I_residue_j,I_restored_j,I_convmodel_j, _, _, _, _, _, _, _, _ , _, _, _, _ = CLEAN_JOINT(I,Q,U,V,I_PSF,mode="I")

In [ ]:
ch = 1
F = plt.figure(1,(15,15))
grid = ImageGrid(F, 111,  # similar to subplot(111)
                nrows_ncols=(2, 2),
                direction="row",
                axes_pad=0.5,
                add_all=True,
                label_mode="1",
                share_all=True,
                cbar_location="right",
                cbar_mode="each",
                cbar_size="3%")

grid[0].set_title("I Restored")
im = grid[0].imshow(I_restored_j[ch], interpolation="nearest", cmap="cubehelix")
grid[0].cax.colorbar(im)
grid[1].set_title("I Residue")
im = grid[1].imshow(I_residue_j[ch], interpolation="nearest", cmap="cubehelix")
grid[1].cax.colorbar(im)
grid[2].set_title("I Model")
im = grid[2].imshow(I_clean_j[ch], interpolation="nearest", cmap="cubehelix")
grid[2].cax.colorbar(im)
grid[3].set_title("I convolved clean map")
im = grid[3].imshow(I_convmodel_j[ch], interpolation="nearest", cmap="cubehelix")
grid[3].cax.colorbar(im)
plt.show()

# Multi-Frequency-Synthesis

## Iterative $\chi^2$ minimisation

The imaging problem can be formulated as an iterative $\chi^2$ minimisation of a system of linear equations of the form
$$ Ax = b + \epsilon,$$
where $\epsilon$ is iid Gaussian noise. This allows us to form the $\chi^2$ distribution as
$$ \chi^2 = (Ax - b)^\dagger W (Ax - b), $$
where $W$ is a diagonal matrix of weights and the superscript $\dagger$ denotes the Hermitian conjugate. The assumption that $\epsilon$ is Gaussian noise is quite important here. Basically it ensures that the quantity we are optimising only has a single stationary point. In contrast, if $\epsilon$ was not Gaussian but rather some multi-modal distribution, then the quantity $\epsilon^\dagger \epsilon$ could also have a multi-modal distribution. 

The goal is to find the set of parameters $x$ which minimises $\chi^2$. The gradient of $\chi^2$ is given by
$$ \partial_x \chi^2 = J(x) = A^\dagger W (b - Ax), $$
where we have used the standard linear algebra notation $J(x)$ to denote the Jacobian (or, since $\chi^2$ is scalar, also the gradient). One more derivative gives the Hessian $H(x)$ as 
$$ \partial^2_x \chi^2 = H = A^\dagger W A. $$
An iterative solution can therefore be obtained from an initial guess, $x_0$ say, using a numerical root finding algorithm such as Newton's method i.e.
$$ x_{i+1} = x_i + H^{-1}(x_i) J(x_i) $$
Furthermore, setting the gradient to zero produces the normal equations
$$ A^\dagger W A x = A^\dagger W b$$
Note that if the Hessian matrix $H = A^H W A$ was non-singular we could solve for $x$ directly from the normal equations
$$ x = \left( A^\dagger W A  \right)^{-1} A^\dagger W b. $$
Unfortunately, because of incomplete $uv$-coverage, in interferometry the Hessian matrix (viz. $(SF)^\dagger W SF$ see below) will almost always be singular ($S$ is rank deficient). This necessitates an iterative solution. Note that the solution to the normal equations is not, in general, unique. The solution is ambiguous w.r.t. the addition of any vector $\tilde{x}$ in the null space of the Hessian i.e. $H\tilde{x} = 0$. The reason for this is that, according to the normal equations, we have
$$ H(x + \tilde{x}) = Hx = A^\dagger W b, $$
regardless of the choice of $\tilde{x}$. 

## Imaging as an iterative $\chi^2$ minimisation problem

The imaging problem can be formulated as an iterative $\chi^2$ minimisation of a system of linear equations of the form
$$ S F I = V_{obs} + \epsilon,$$
where $S$ is the degridding operator, $F$ the Fourier transform and $I$ is the image we want to deconvolve. In this case we form the $\chi^2$ distribution as
$$ \chi^2 = (S F I - V_{obs})^\dagger W (S F I - V_{obs}), $$
where $W$ are the imaging weights. The goal is to find the image $I$ which minimises $\chi^2$. 
In this case the gradient is given by
$$ \partial_x \chi^2 = J(I) = F^\dagger S^\dagger W (S F I - V_{obs}), $$
and the Hessian is 
$$ \partial^2_x \chi^2 = H = F^\dagger S^\dagger W S F. $$

An iterative solution can therefore be obtained from an initial guess, $I_{0}$ say, using a numerical root finding algorithm such as Newton's method i.e.
$$ I_{i+1} = I_i + g(H^{-1}) J(I_i), $$
where $g$ is a function which approximates the operator $H^{-1}$ and can also be used to control the step size. As already mentioned above, the normal equations viz.
$$F^\dagger S^\dagger W SF I = F^\dagger S^\dagger W V_{obs} = I_D $$
where $I_D$ denotes the dirty image, can't be inverted for $I$ in a straighforward manner and we need to use the iterative approach. In imaging the iterative solution is usually implemented using major and minor cycles. Lets see how this works. Firstly, note that, for a given model image $I^{M}$, the Jacobian is just the residual image i.e. 
$$ J(I^{M}) = F^\dagger S^\dagger W (S F I^M - V_{obs}) = I^{R}. $$
Thus one iteration of the optimisation method would be given by 
$$ I^M_{i+1} = I^M_{i} + g(H^{-1}) I^R_{i}. $$
The fact that $H$ is singular makes this step a bit tricky to implement directly. Recall that the solution $H^{-1} I^R$ represents a deconvolution of $I^{PSF} * I$. The solution can be approximated via the function $g(\cdot)$ but, in general, this requires a number of assumptions. We will not go into the details but simply outline how the function is constructed. Assuming that the PSF's are delta functions results in a diagonal Hessian matrix. If they are also spatially invariant then all the elements on the diagonal are the same. Thus the Hessian reduces to a single number viz. the value of the PSF at the center (note we are assuming that neither the residual image nor the PSF has been normalised). We can then start building up the model image by searching for the peak in $I^R$, multiplying by $g(H^{-1})$ (equivalently normalising $I^R$ by the value of the PSF at the center) and adding the result to $I^M$. We can then compute the new residual image (i.e. re-evaluate the Jacobian) and repeating until some prespecified convergence criterion has been reached. 

Since evaluating the Jacobian is an expensive operation, the algorithm is usually implemented slightly differently. This is where the distinction between minor and major cycles comes in. Major cycles evaluate the Jacobian while minor cycles build up the model image. It is impractical to, within any given minor cycle, build the model image one pixel at a time. To speed things up a bit we might want to add multiple pixels (the brightest ones) to the model image during the minor cycle. This requires performing the subtraction (equivalently computing the Jacobian) in the image domain and can be implemented approximately by subtracting the PSF centered at the location of the current brightest pixel from the entire image. For such an approach it is also advantages to normalise both $I^R$ and $I^{PSF}$ by the value of the PSF at the center. In this case the intensities in $I^D$ (i.e. the first residual image) corresponde to those of $I^M$. Below we give a quick and dirty implementation of this iterative optimisation algorithm.

In [ ]:
def CLEAN_CS(Vobs,lmn,K,invK,PSF,
             gamma = 0.1,threshold = "Default", nminoriter = 50,
             nmajoriter = 20):
    """
    This is a basic CLEAN implemented using major and minor cycles. 
    Input:  Vobs = precalibrated visibilities
            lmn = the coordinates of the image
            K = the DFT kernel
            invK = the inverse DFT kernel
            PSF = the unnormalised PSF
    """
    #Initialise model image
    n = Vobs.shape[0]
    Vobs = Vobs.reshape(n,1)
    npix = int(np.sqrt(K.shape[1]))
    m = npix*npix
    IM = np.zeros([npix,npix])
    
    #Normalise PSF keeping track of maximum
    PSF_max = np.max(PSF)
    PSF /= PSF_max
    
    #Get dirty image
    print "Doing iDFT"
    ncoord = (lmn[:,-1] + 1.0).reshape(m,1)
    ID = np.real(ncoord*np.dot(invK,Vobs)/PSF_max).reshape(npix,npix) #ncoord*
    print ID.shape
    
    #Find first maximum
    p,q = (argwhere(ID == ID.max())[0]).squeeze()
    Istar = ID[p,q]
    
    #Set threshold
    if threshold=="Default":
        threshold = 0.1*np.abs(Istar) #Imin + 0.001*(Istar - Imin)
        print "Threshold set at ", threshold
    else:
        print "Assuming user set threshold"
        
    #Start deconvolution
    i = 0
    cont = True
    IR = ID.copy()
    while (i < nmajoriter) and cont:
        #Enter minor cycle
        j = 0
        while (j < nminoriter) and (Istar > threshold):
            #Update model image
            IM[p,q] += gamma*Istar
            
            #Do image plane subtraction
            IR -= gamma*Istar*PSF[npix - p:2*npix - p,npix - q:2*npix - q] 
    
            #Get new indices where IR is max
            p,q = (argwhere(IR == IR.max())[0]).squeeze()
            Istar = IR[p,q] 
            
            #Increment minor cycle counter
            j += 1 
        #Increment major cycle counter
        i += 1
        print i
        #Get residual image and check convergence criterion
        IR = get_Jacobian(Vobs,IM,K,invK,ncoord,npix,PSF_max)
        p,q = (argwhere(IR == IR.max())[0]).squeeze()
        Istar = IR[p,q]        
        if Istar < threshold:
            cont = False
            
    #Warn if number of iterations exceed
    if i >= nmajoriter:
        print "Max iterations exceeded. Istar = ",Istar

    return IM
    
    
def get_Jacobian(Vobs,IM,K,invK,ncoord,npix,PSF_max):
    #Apply DFT to I
    Vpred = np.dot(K,IM.reshape(npix*npix,1)/ncoord)
    
    #Get residual vis
    Vres = Vobs - Vpred
    
    #Get IR
    IR = np.real(ncoord*np.dot(invK,Vres)/PSF_max)
    return IR.reshape(npix,npix)

In [ ]:
#Here we do the CS CLEAN over a single channel
#Load in MS
ms = tbl("DATA/polarized_sky.MS_p0/")

#Get freq info
msfreq = tbl("DATA/polarized_sky.MS_p0::SPECTRAL_WINDOW")

#Get pointing center
msfield = tbl("DATA/polarized_sky.MS_p0::FIELD")
ra0, dec0 = msfield.getcol("PHASE_DIR").squeeze() #in radians

#Some constants and conversion factors
c = 2.99792458e8 #speed of light
npix = 128 #1025
ARCSEC2RAD = 4.8481e-6
delta_pix = 2 * ARCSEC2RAD * 2 * 2 * 2 
uv_scale = npix * delta_pix

#Select subset of data
nchan = 5 #-1 to select all (NOTE this selects all but the last)
nrows = 1000 #-1 to select all
uvw = ms.getcol("UVW")[0:nrows,:] 
vis = ms.getcol("DATA")[0:nrows,0:nchan,:]
#Delete autocorelations
I = np.argwhere(uvw[:,0] == 0.0).squeeze()
uvw = np.delete(uvw,I,axis=0)
scaled_uvw = uvw * uv_scale
vis = np.delete(vis,I,axis=0)

#Get frequencies
Freqs = msfreq.getcol("CHAN_FREQ").squeeze()[0:nchan]
ref_freq = Freqs[0]

#Close tables
ms.close()
msfreq.close()
msfield.close()

#Get wavelengths
ref_lambda = c / Freqs

In [ ]:
#Set ra and dec
ra = ra0 + np.linspace(ra0 - npix*delta_pix/2.0,ra0 + npix*delta_pix/2.0,npix)
#ra_PSF  = ra0 + np.linspace(ra0 - npix*delta_pix,ra0 + npix*delta_pix,2*npix)
delta_ra = ra - ra0
#delta_ra_PSF = ra_PSF - ra0
dec = dec0 + np.linspace(dec0 - npix*delta_pix/2.0,dec0 + npix*delta_pix/2.0,npix)
#dec_PSF = dec0 + np.linspace(dec0 - npix*delta_pix,dec0 + npix*delta_pix,2*npix)

#Get corresponding l and m
l = (np.cos(dec)*np.sin(delta_ra))
#l_PSF = (np.cos(dec_PSF)*np.sin(delta_ra_PSF))
m = (-np.sin(dec)*np.cos(dec0) - np.cos(dec)*np.sin(dec0)*cos(delta_ra))
#m_PSF = (-np.sin(dec_PSF)*np.cos(dec0) - np.cos(dec_PSF)*np.sin(dec0)*cos(delta_ra_PSF))

print "Getting lmn"
lmn = get_lmn(l,m)
n = lmn[:,2] + 1.0
#lmn_PSF = get_lmn(l_PSF,m_PSF)
#n_PSF = lmn_PSF[:,2] + 1.0

#print "Getting DFT kernels for PSF"
#invK_PSF = set_kernels(2*npix,2*npix,lmn_PSF,uvw,ref_lambda)

#print "Computing PSF"
#PSF = iDFT(np.ones([vis.shape[0],nchan]),invK_PSF,n_PSF) #np.abs(dot(invK[0,:,:],np.ones(vis.shape[0])))
#PSF = np.abs(PSF.reshape(nchan,2*npix,2*npix))
#PSF0 = PSF[0]

#del(invK_PSF)
#del(l_PSF)
#del(m_PSF)
#del(lmn_PSF)
#del(n_PSF)

print "Getting DFT kernels"
K = set_kernels(npix,npix,lmn,uvw,ref_lambda)
#invK0 = invK[0]
#K0 = invK0.conjugate().T
#K = np.zeros([nchan,uvw.shape[0],npix*npix],dtype=complex)
#for i in range(nchan):
#    K[i,:,:] = invK[i,:,:].conjugate().T

#Get V corresponding to stokes I 
Vobs = (vis[:,:,0] + vis[:,:,3])*0.5

#Create gridder object
gridder = grid_it(scaled_uvw,ref_lambda,npix,npix,aa)

print "Done"

In [ ]:
print "Doing CS CLEAN"
IM = CLEAN_CS(Vobs[:,0].copy(),lmn,K[0].copy(),invK[0].copy(),PSF[0].copy())

In [ ]:
plt.imshow(IM,interpolation="nearest", cmap="cubehelix")
plt.colorbar()

# MFS

Start by Taylor expanding the model image $I^M(\nu)$ into 
$$ I^M(\nu) = \sum_{t=0}^{N_t-1} w(\nu)^t I^T_t, $$
where $I^T_t = \frac{d^t}{d \nu^t}I^M(\nu)|_{\nu = \nu_0}$ is the $t^{th}$ Taylor coefficient and $w(\nu) = \left(\frac{\nu - \nu_0}{\nu_0}\right)$ can be thought of as basis functions. Substitute this into the expression for $V^{obs}$ to find
$$ V^{obs}_\nu = \sum_{t = 0}^{N_t-1} w(\nu)^t S_\nu F I^T_t = 
\left[ \begin{array}{cccc}
[ w^0_v S_v F] & [ w^1_v S_v F] & \cdot & \cdot
\end{array} \right] \left[\begin{array}{c} I^T_0 \\ I^T_1 \\ \cdot \\ \cdot \end{array} \right], $$
where in the last step we perform the summation using block matrices. MFS basically works by using all the visibility data (i.e. in each channel/band) to estimate the Taylor components $I^T$. This can be achieved by solving the following system
$$
\left[ \begin{array}{cccc}
[ w^0_{v_1} S_{v_1} F] & [ w^1_{v_1} S_{v_1} F] & \cdot & \cdot \\
[ w^0_{v_2} S_{v_2} F] & [ w^1_{v_2} S_{v_2} F] & \cdot & \cdot \\
\cdot & \cdot & \cdot & \cdot
\end{array} \right] \left[\begin{array}{c} I^T_0 \\ I^T_1 \\ \cdot \\ \cdot \end{array} \right] = \left[\begin{array}{c} V^{obs}_{\nu_1} \\ V^{obs}_{\nu_2} \\ \cdot \\ \cdot \end{array} \right].
$$
The normal equations for this system can be found simply by multiplying through by the Hermitian conjugate of the matrix on the left
$$
\left[ \begin{array}{ccc}
[ w^0_{v_1} S^\dagger_{v_1} F^\dagger] & [w^0_{v_2} S^\dagger_{v_2} F^\dagger] & \cdot \\
[ w^1_{v_1} S^\dagger_{v_1} F^\dagger] & [ w^1_{v_2} S^\dagger_{v_2} F^\dagger] & \cdot \\
\cdot & \cdot & \cdot  \\
\cdot & \cdot & \cdot 
\end{array} \right]
\left[ \begin{array}{cccc}
[ w^0_{v_1} S_{v_1} F] & [ w^1_{v_1} S_{v_1} F] & \cdot & \cdot \\
[ w^0_{v_2} S_{v_2} F] & [ w^1_{v_2} S_{v_2} F] & \cdot & \cdot \\
\cdot & \cdot & \cdot & \cdot
\end{array} \right] \left[\begin{array}{c} I^T_0 \\ I^T_1 \\ \cdot \\ \cdot \end{array} \right] = 
\left[ \begin{array}{ccc}
[ w^0_{v_1} S^\dagger_{v_1} F^\dagger] & [w^0_{v_2} S^\dagger_{v_2} F^\dagger] & \cdot \\
[ w^1_{v_1} S^\dagger_{v_1} F^\dagger] & [ w^1_{v_2} S^\dagger_{v_2} F^\dagger] & \cdot \\
\cdot & \cdot & \cdot  \\
\cdot & \cdot & \cdot 
\end{array} \right]
\left[\begin{array}{c} V^{obs}_{\nu_1} \\ V^{obs}_{\nu_2} \\ \cdot \\ \cdot \end{array} \right].
$$
or
$$ H I^T = I, $$
where the images on the RHS are constructed from $I^D_{\nu}$ according to
$$ I_i = \sum_{\nu_k} w_{\nu_k}^i S^\dagger_{v_k} F^\dagger V^{obs}_{\nu_k} = \sum_{\nu_k} w_{\nu_k}^i I^D_{\nu_k}, $$
and the Hessian matrix consists of block matrices of the following form
$$ H_{ij} = \sum_{\nu_k} w_{\nu_k}^{i+j} F^\dagger S^\dagger_{\nu_k} S_{\nu_k} F. $$ 
The solution to the normal equations therefore provides the Taylor components $I^T$. The full Hessian matrix will not, in general, be invertible. However, by making the same assumptions as before, we can reduce each $H_{ij}$ to a single number viz. 
$$ H_{ij} = \sum_{\nu_k} w_{\nu_k}^{i + j} \mbox{mid}(I^{PSF}_{\nu_k}). $$
The deconvolution can be approximated by searching for the peak in $I_0$ (since $I_0 = \sum_{\nu_k} I^D_{\nu_k}$ this is similar to searching the average image as is done in joint deconvolution) and computing $I^T$ using
$$ I^T = H^{-1}I $$
These can now be used to compute the predicted visibilities required to compute the Jacobian in the major cycle. Subtraction within the image domain is slightly trickier. We have to get
$$ I^M(\nu) = \sum_{t=0}^{N_t-1} w(\nu)^t I^T_t $$
for each frequency and subtract $I^M * I^{PSF}$ from $I^D$ and use the updated $I^D$ to again get $I$ (there might be a simpler way to do this). 

Assuming a spectral index model
$$ I^M(v) = I^M_0 (\frac{\nu}{\nu_0})^\alpha $$
shows that
$$ I^T_0 = I^M_0, \quad I^T_1 = \alpha I^M_0, $$
so that we get an alpha map from
$$ \alpha = \frac{I^T_1}{I^T_0}. $$

In [ ]:
def CLEAN_MFS(Vobs,lmn,K,ID,PSF,Nt,Freqs,ref_freq,gridder,
             gamma = 0.25,threshold = "Default", nminoriter = 10,
             nmajoriter = 100):
    """
    This is a basic MFS CLEAN implemented using major and minor cycles. 
    Input:  Vobs = precalibrated visibilities
            lmn = the coordinates of the image
            K = the DFT kernel
            invK = the inverse DFT kernel
            PSF = the unnormalised PSF
    """
    #Initialise model image
    n = Vobs.shape[0]
    nchan = Vobs.shape[1]
    npix = int(np.sqrt(K.shape[2]))
    m = npix*npix
    IT = np.zeros([Nt,npix,npix])
    IM = np.zeros([nchan,npix,npix])
    
    #Normalise PSF keeping track of maximum
    PSF_max = np.zeros(nchan)
    for i in xrange(nchan):
        PSF_max[i] = np.max(PSF[i])
        PSF[i] /= PSF_max[i]
    print PSF_max
    #Get dirty image
    print "Getting Dirty cube"
    ncoord = (lmn[:,-1] + 1.0).reshape(m,1)

    #Compute weighted sum of dirty's
    print "Getting I from ID"
    I = set_Dirty_Comps(ID,Freqs,ref_freq,Nt)
    
    Itmp = I[0] #this is where we do the peak finding
    
    #Find first maximum
    p,q = (argwhere(Itmp == Itmp.max())[0]).squeeze()
    Istarm = Itmp[p,q]
    
    #Set the Hessian and get inverse
    print "Setting H"
    H = set_Hessian(PSF_max,Freqs,ref_freq,Nt,nchan)
    invH = np.linalg.inv(H)
    print invH
    
    #Solve for Taylor components
    Istar = np.dot(invH,I[:,p,q])
    
    #Set threshold
    if threshold=="Default":
        threshold = 0.2*np.abs(Istarm) 
        print "Threshold set at ", threshold
    else:
        print "Assuming user set threshold"
        
    #Start deconvolution
    i = 0
    IR = ID.copy()
    print "Deconvolving"
    while (i < nmajoriter) and (Istarm > threshold):
        #Increment major cycle counter
        i += 1
        if i%10 == 0:
            print i, Istarm, Istar.max()
        
        #Update IT
        IT[:,p,q] += gamma*Istar
        
        #Get residual image
        IR, I = get_Jacobian_MFS(Vobs,IT,K,ncoord,npix,PSF_max,Freqs,ref_freq,gridder)
        
        #Find next peak
        Itmp = I[0]
        p,q = (argwhere(Itmp == Itmp.max())[0]).squeeze()
        Istarm = Itmp[p,q]
        
        #get IT
        Istar = np.dot(invH,I[:,p,q])
    
            
    #Warn if number of iterations exceeded
    if i >= nmajoriter:
        print "Max iterations exceeded. Istar = ",Istarm
    
    print "Done"
    
    #Get IM
    w = (Freqs - ref_freq)/ref_freq
    for k in xrange(nchan):
        for l in xrange(Nt):
            IM[k] += w[k]**l*IT[l] #/mp.factorial(l)
    
    #Get alpha and beta maps
    alpha = IT[1]/IT[0]
    return IT, IM, ID, IR, alpha

def get_Jacobian_MFS(Vobs,IT,K,ncoord,npix,PSF_max,Freqs,ref_freq,gridder):
    #Get number of channels/bands
    n = Vobs.shape[0]
    nchan = Freqs.size
    Nt = IT.shape[0]
    
    #Apply weighted DFT to each IT
    Vpred = np.zeros_like(Vobs)
    w = (Freqs - ref_freq)/ref_freq
    for i in xrange(nchan):
        for j in xrange(Nt):
            Vpred[:,i] += w[i]**j*np.dot(K[i],IT[j].reshape(npix*npix,1)/ncoord).squeeze()
                
    #Get residual vis
    Vres = Vobs - Vpred
    
    #Get residual image
    IR = np.real(gridder.give_IR(Vres))
    
    #Get I
    I = set_Dirty_Comps(IR,Freqs,ref_freq,Nt)
    return IR, I

def set_Hessian(PSF_max,Freqs,ref_freq,Nt,nchan):
    """
    Constructs principal components of the Hessian
    """
    #Create array to store Hessian
    H = np.zeros([Nt,Nt])
    #Set basis function
    w = (Freqs - ref_freq) / ref_freq
    #Compute Hessian
    for i in xrange(Nt):
        for j in xrange(Nt):
            tmp = 0.0
            for k in range(nchan): 
                tmp += w[k]**(i+j)*PSF_max[k]
            H[i,j] = tmp
    return np.diag(np.diag(H))

def set_Dirty_Comps(ID,Freqs,ref_freq,Nt):
    nchan,nx,ny = ID.shape
    I = np.zeros([Nt,nx,ny])
    tmp = np.zeros([nx,ny])
    w = (Freqs - ref_freq) / ref_freq
    for t in xrange(Nt):
        for i in xrange(nchan):
            tmp += w[i]**t*ID[i,:,:]
        I[t,:,:] = tmp
    return I

In [ ]:
Nt = 3
IT, IM, ID, IR, alpha = CLEAN_MFS(Vobs.copy(),lmn.copy(),K.copy(),ID2.copy(),psf2.copy(),Nt,Freqs,ref_freq,gridder)

In [ ]:
plt.imshow(IT[2],interpolation="nearest", cmap="cubehelix")
plt.colorbar()

In [ ]:
        #Enter minor cycle
        j = 0
        while (j < nminoriter) and (Istarm > threshold):
            #Update model image
            IM[:,p,q] += gamma*Istar
            
            #Do image plane subtraction
            IR -= gamma*Istar[:,np.newaxis,np.newaxis]*PSF[:,npix - p:2*npix - p,npix - q:2*npix - q] 
            
            #Get new I
            I = set_Dirty_Comps(IR,Freqs,ref_freq,Nt)
            Itmp = I[0]
            
            #Get new indices where I is max
            p,q = (argwhere(Itmp == Itmp.max())[0]).squeeze()
            Istarm = Itmp[p,q]
            
            #Solve of IT
            IT = np.dot(invH,I[:,p,q])
            
            for k in xrange(nchan):
                for l in xrange(Nt):
                    Istar[k] += w[k]**l*IT[l]
            
            #Increment minor cycle counter
            j += 1 

            
def get_Jacobian_MFS(Vobs,IM,K,invK,ncoord,npix,PSF_max,Freqs,ref_freq):
    #Get number of channels/bands
    n = Vobs.shape[0]
    nchan = Freqs.size
    
    #Apply weighted DFT to each IM
    Vpred = np.zeros_like(Vobs)
    for i in xrange(nchan):
            Vpred[:,i] = np.dot(K[i],IM[i].reshape(npix*npix,1)/ncoord).squeeze()
                
    #Get residual vis
    Vres = Vobs - Vpred
    
    #Get IR
    IR = np.zeros([nchan,npix,npix])
    for i in xrange(nchan):
        IR[i] = np.real(ncoord*np.dot(invK[i],Vres[:,i].reshape(n,1))/PSF_max[i]).reshape(npix,npix)
    
    #Get I
    I = set_Dirty_Comps(IR,Freqs,ref_freq,Nt)
    return IR, I

In [ ]:
def diag_dot(A,B):
    D = np.zeros(A.shape[0],dtype=complex)
    for i in range(A.shape[0]):
        D[i] = dot(A[i,:],B[:,i])
    return D    

In [ ]:
Nt = 3
nchan = 5
w = (Freqs - ref_freq)/ref_freq
H = np.zeros([Nt,Nt])
PSF_max = np.zeros(nchan)
for i in range(nchan):
    PSF_max[i] = np.abs(diag_dot(K[i],invK[i])).max()

for i in range(Nt):
    for j in range(Nt):
        tmp = 0.0
        for k in range(nchan):
            tmp += w[k]**(i+j)*PSF_max[k]
        H[i,j] = tmp

print PSF_max        
print np.linalg.inv(H)

In [ ]:
print lmn

In [ ]:
Freqs = ref_freq
ref_freq = Freqs[0]